In [1]:
import pandas as pd
import numpy as np
import torch
import monai
import torchvision
import numpy as np
import os
import shutil
import matplotlib
from PIL import Image
from tqdm import tqdm
import os
from glob import glob
from torch.utils.data import DataLoader, Dataset



Duplicate key in file '/Users/tobymao/.matplotlib/matplotlibrc', line 2 ('backend: TkAgg')


In [49]:
import nibabel as nib
#create a dataloader 

class imagedata(Dataset):
    def __init__(self, img_dir, label_dir, transform= None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images_list = sorted(os.listdir(img_dir))
        self.masks_list = sorted(os.listdir(label_dir))
        self.patients_list = sorted(set('_'.join(f.split('_')[:-1]) for f in os.listdir(img_dir) if f.endswith('.nii')))
    def __getitem__(self, index):
        patient_id = self.patients_list[index]
        t1_name = os.path.join(self.img_dir, f"{patient_id}_t1.nii")
        flair_name = os.path.join(self.img_dir, f"{patient_id}_flair.nii")
        t2_name = os.path.join(self.img_dir, f"{patient_id}_t2.nii")
        mask_name = os.path.join(self.label_dir, f"{patient_id}_seg.nii")
        t1_image = nib.load(t1_name).get_fdata()
        flair_image = nib.load(flair_name).get_fdata()
        t2_image = nib.load(t2_name).get_fdata()
        mask = nib.load(mask_name).get_fdata()

        sample = {
            'image': {
                't1': t1_image,
                'flair': flair_image,
                't2': t2_image
            },
            'mask': mask
        }

        
        if self.transform:
            sample = self.transform(sample)

        return sample['image'], sample['mask']

        
    def __len__(self):
        return len(self.patients_list)


In [50]:
#check and see if the dataloader function works
dataset = imagedata(img_dir='/Volumes/toby/BRATS2017/TrainingDataset/images', label_dir='/Volumes/toby/BRATS2017/TrainingDataset/labels')
index = 0 
image,mask = dataset.__getitem__(index)


print(len(image['t1']), len(image['t2']), len(image['flair']))
print(len(mask))

240 240 240
240


In [52]:
from torchvision import transforms
class ToTensorAndNormalize:
    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        
        # Convert to tensor
        t1 = torch.tensor(image['t1'], dtype=torch.float32)
        flair = torch.tensor(image['flair'], dtype=torch.float32)
        t2 = torch.tensor(image['t2'], dtype=torch.float32)
        mask = torch.tensor(mask, dtype=torch.float32)

        # Normalize images
        t1 = (t1 - t1.mean()) / t1.std()
        flair = (flair - flair.mean()) / flair.std()
        t2 = (t2 - t2.mean()) / t2.std()

        return {
            'image': {
                't1': t1,
                'flair': flair,
                't2': t2
            },
            'mask': mask
        }


In [57]:
# Instantiate the dataset with the custom transform
transform = transforms.Compose([ToTensorAndNormalize()])

dataset = imagedata(img_dir='/Volumes/toby/BRATS2017/TrainingDataset/images', label_dir='/Volumes/toby/BRATS2017/TrainingDataset/labels')
index = 0  
image, mask = dataset[index]

print(image)
print(mask)
print(image['t1'].shape)

{'t1': memmap([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.

In [58]:
"""from skimage import imread
# create a new data loader
class BrainSegmentationDataset(Dataset):
    in_channels = 3
    out_channels = 1

    def __init__(
        self,
        images_dir,
        mask_dir,
        image_type,
        transform=None,
        image_size=256,
        subset="train",
        random_sampling=True,
        validation_cases=10,
        seed=42,
    ):
        assert subset in ["all", "train", "validation"]
        self.images_list = sorted(os.listdir(images_dir))
        self.masks_list = sorted(os.listdir(mask_dir))
        self.patients_list = sorted(set('_'.join(f.split('_')[:-1]) for f in os.listdir(images_dir) if f.endswith('.nii')))
        # read images
        volumes = {}
        masks = {}
        t1_name = [os.path.join(images_dir, f"{i}_t1.nii") for i in self.patients_list]
        flair_name = [os.path.join(images_dir, f"{i}_flair.nii") for i in self.patients_list]
        t2_name = [os.path.join(images_dir, f"{i}_t2.nii") for i in self.patients_list]
        mask_name = [os.path.join(mask_dir, f"{i}_seg.nii") for i in self.patients_list]
        print("reading {} images...".format(subset))
        if image_type == 't1':
            for a,b in zip(t1_name,self.patients_list):
                image_slice = []
                image_slice.append(imread(a))
            volumes[b] = np.array(image_slice[1:-1])
        elif image_type == 't2':
            for a, b in zip(t2_name, self.patients_list):
                image_slice = []
                image_slice.append(imread(a))
            volumes[b] = np.array(image_slice[1:-1])
        elif image_type == 'flair':
            for a,b in zip(flair_name, self.patients_list):
                image_slice = []
                image_slice.append(imread(a))
            volumes[b] = np.array(image_slice[1:-1])
        for c,d in zip(mask_name, self.patients_list):
            mask_slice = []
            mask_slice.append(imread(c))
        masks[d] = np.array(mask_slice[1:-1])"""
            
        






OptionalImportError: from einops.layers.torch import Rearrange (No module named 'einops').

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies

In [26]:
"""
import os
import numpy as np
import nibabel as nib

images_dir = "/Volumes/toby/BRATS2017/TrainingDataset/images"
mask_dir = '/Volumes/toby/BRATS2017/TrainingDataset/labels'

# List and sort all files in the images and masks directories
images_list = sorted(os.listdir(images_dir))
masks_list = sorted(os.listdir(mask_dir))

# Extract unique patient IDs
patients_list = sorted(set('_'.join(f.split('_')[:-1]) for f in images_list if f.endswith('.nii')))

# Define file paths for different image types
t1_name = {patient: os.path.join(images_dir, f"{patient}_t1.nii") for patient in patients_list}
flair_name = {patient: os.path.join(images_dir, f"{patient}_flair.nii") for patient in patients_list}
t2_name = {patient: os.path.join(images_dir, f"{patient}_t2.nii") for patient in patients_list}
mask_name = {patient: os.path.join(mask_dir, f"{patient}_seg.nii") for patient in patients_list}

# Initialize dictionaries for volumes and masks
volumes = {}
masks = {}

# Load images based on the specified image type
image_type = 't1'
image_files = t1_name if image_type == 't1' else t2_name if image_type == 't2' else flair_name

for patient, filepath in image_files.items():
    volumes[patient] = imread(filepath)

# Load mask images
for patient, filepath in mask_name.items():
    masks[patient] = imread(filepath)

# Processed volumes and masks now contain the 3D numpy arrays"""


KeyboardInterrupt: 

In [27]:

from monai.transforms import (
    Compose,
    Resized,
    ToTensord,
    LoadImaged,
    MapTransform,
    NormalizeIntensityd,
    Orientationd,
    RandFlipd,
    RandShiftIntensityd,
    Spacingd,
    EnsureChannelFirstd,
    RandSpatialCropd,
    RandScaleIntensityd
)
from monai.apps import DecathlonDataset
from monai.data import DataLoader
